In [53]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split as tts

In [54]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 hours 59 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,"7 days, 20 hours and 45 minutes"
H2O_cluster_name:,H2O_from_python_pomilusky_uk74x7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.659 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [55]:
df = pd.read_csv('../Data/new_train.csv')
train, test = tts(df, test_size=0.3, train_size= 0.7, random_state = 7)
train.to_csv('../Data/h20_train.csv')
test.to_csv('../Data/h20_test.csv')

In [56]:
train = h2o.import_file('../Data/h20_train.csv')
test = h2o.import_file('../Data/h20_test.csv')


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [57]:
X =train.columns
y = 'price'
X.remove(y)

In [43]:
# train[y]=train[y].asfactor()
# test[y]=test[y].asfactor()

In [58]:
aml=H2OAutoML()
aml.train(x=X, y=y, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

15:37:00.684: GBM_lr_annealing_selection_AutoML_8_20220224_143731 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_8_20220224_143731

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 1219.2277909666875
RMSE: 34.917442503234504
MAE: 17.18518207856428
RMSLE: 0.20588613731552724
R^2: 0.9086770829267786
Mean Residual Deviance: 1219.2277909666875
Null degrees of freedom: 2646
Residual degrees of freedom: 2642
Null deviance: 35339387.59425765
Residual deviance: 3227295.962688822
AIC: 26333.371070464957

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 4140.194429289391
RMSE: 64.34434263623642
MAE

In [59]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_8_20220224_143731

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 1219.2277909666875
RMSE: 34.917442503234504
MAE: 17.18518207856428
RMSLE: 0.20588613731552724
R^2: 0.9086770829267786
Mean Residual Deviance: 1219.2277909666875
Null degrees of freedom: 2646
Residual degrees of freedom: 2642
Null deviance: 35339387.59425765
Residual deviance: 3227295.962688822
AIC: 26333.371070464957

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 4140.194429289391
RMSE: 64.34434263623642
MAE: 28.504609336589827
RMSLE: NaN
R^2: 0.6898900801521026
Mean Residual Deviance: 4140.194429289391
Null degrees of freedom: 2646
Residual degrees of freedom: 2642
Null deviance: 35345505.665858395
Residual deviance: 10959094.654329019
AIC: 29569.39487837725


In [60]:
test = h2o.import_file('../Data/new_test.csv')
pred = aml.leader.predict(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/home/pomilusky/miniconda3/envs/ironhack/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column 'C1': substituting in a column of NaN
  warnings.warn(w)
/home/pomilusky/miniconda3/envs/ironhack/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'host_neighbourhood' has levels not trained on: ["Belváros", "Nieuwendammerham"]
  warnings.warn(w)


In [61]:
pred = pred.as_data_frame()

In [80]:
#test['price'] = pred
df_sample = pd.read_csv('../Data/sample.csv')
df_sample['price']=pred
df_sample.head()

,price,id
0,129.696615,0
1,67.882471,1
2,106.318701,2
3,128.340921,3
4,124.578182,4


In [86]:
df_sample.to_csv('../Data/resultats.csv',index=False)

In [82]:
test = pd.read_csv('../Data/new_test.csv')
test['price'] = pred
test_sp = pd.read_csv('../Data/resultatsambprice.csv')
test_sp.head()
test['price_nd'] = test_sp['price']

In [83]:
a = test[['price_now','price','price_nd']].reset_index(drop=True)

In [84]:
a[test.price_now.isna()]

,price_now,price,price_nd
0,NaN,129.696615,141.977830
7,NaN,214.860479,226.913891
51,NaN,202.331933,230.806434
57,NaN,76.796840,61.688450
59,NaN,164.400424,169.343025
...,...,...,...
1344,NaN,131.880743,140.102405
1360,NaN,200.554758,210.075836
1362,NaN,74.876882,99.194002
1374,NaN,227.485583,226.388374


In [85]:
test_sp = pd.read_csv('../Data/resultats.csv')
test_sp.head()

,price,id
0,177.264127,0
1,127.178101,1
2,134.649070,2
3,177.779191,3
4,147.028703,4


### Best results:
MSE: 5034.07568408847   
RMSE: 70.95122045524283  
MAE: 32.03218826345224  
RMSLE: NaN  
R^2: 0.6305737999847885  
Mean Residual Deviance: 5034.07568408847  
Null degrees of freedom: 3024  
Residual degrees of freedom: 3022  
Null deviance: 41234721.48402415  
Residual deviance: 15228078.944367621    
AIC: 34377.63338663671  


### 20 models amb 75.25
MSE: 4413.113506239728  
RMSE: 66.43126903981083  
MAE: 28.025055732584413    
RMSLE: 0.299146877411435  
R^2: 0.6722681750995019  
Mean Residual Deviance: 4413.113506239728  
Null degrees of freedom: 2835  
Residual degrees of freedom: 2830  
Null deviance: 38199287.598680854  
Residual deviance: 12515589.903695868  
AIC: 31862.883490100554  

### 45 models amb .8.2
MSE: 4502.4605954901745  
RMSE: 67.1003770145159  
MAE: 28.233104156429235  
RMSLE: 0.2907156924644855  
R^2: 0.6695864319705112  
Mean Residual Deviance: 4502.4605954901745  
Null degrees of freedom: 3024  
Residual degrees of freedom: 3007  
Null deviance: 41262429.96852899  
Residual deviance: 13619943.301357778  
AIC: 34070.02558496956  

### - models amb .8.2
MSE: 4551.037260240422  
RMSE: 67.4613760624583  
MAE: 28.436830794412877  
RMSLE: 0.2944180061210839  
R^2: 0.6660216280632479  
Mean Residual Deviance: 4551.037260240422  
Null degrees of freedom: 3024  
Residual degrees of freedom: 3020  
Null deviance: 41222699.03765267  
Residual deviance: 13766887.712227277  
AIC: 34076.48725301213  

## - models .7/.3
MSE: 4039.3116292382847  
RMSE: 63.555579056745955  
MAE: 27.384928106351573  
RMSLE: 0.28838987687471507  
R^2: 0.6974464298772651  
Mean Residual Deviance: 4039.3116292382847  
Null degrees of freedom: 2646  
Residual degrees of freedom: 2620  
Null deviance: 35364387.032175496  
Residual deviance: 10692057.88259374  
AIC: 29548.0974603153  